# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [50]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
default_config = LdaGeneratorConfig(corpus_file_path=corpus_path)
lda, dictionary = LdaModelGenerator(default_config).make_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

In [51]:
from gensim.models import CoherenceModel
import pandas as pd

test_ds = pd.read_csv("../dataset/output/default_sentences/pre_processed.80k.test.csv")

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
coh = CoherenceModel(lda, texts=texts, coherence='c_v', topn=topn)
print(f"On topn={topn} we have:")
print(f"Model coherence of: {coh.get_coherence()}")
print("Coherence per topic:")
coh.get_coherence_per_topic()

On topn=10 we have:
Model coherence of: 0.3868902822755468
Coherence per topic:


[0.436156336933028,
 0.3611579852541559,
 0.37026848878042784,
 0.2714778263511153,
 0.41402675610427825,
 0.6580118263437783,
 0.5004098024717523,
 0.3042609099976455,
 0.24026402517496237,
 0.49165048179127374,
 0.32553021339553967,
 0.39583862010431337,
 0.29265365730454596,
 0.3547570218508393]

In [52]:
lda.show_topics(20, 20)

[(0,
  '0.104*"game" + 0.077*"theme" + 0.064*"mechanic" + 0.052*"interesting" + 0.038*"get" + 0.034*"placement" + 0.033*"round" + 0.031*"art" + 0.030*"end" + 0.028*"start" + 0.022*"great" + 0.022*"make" + 0.018*"kind" + 0.018*"piece" + 0.018*"good" + 0.015*"thematic" + 0.015*"story" + 0.014*"abstract" + 0.014*"difficult" + 0.013*"style"'),
 (1,
  '0.419*"<game_name>" + 0.063*"new" + 0.045*"version" + 0.036*"character" + 0.027*"edition" + 0.025*"scenario" + 0.023*"prefer" + 0.022*"play" + 0.022*"map" + 0.020*"definitely" + 0.018*"track" + 0.015*"variety" + 0.014*"add" + 0.014*"game" + 0.013*"fairly" + 0.013*"copy" + 0.012*"fit" + 0.011*"stuff" + 0.010*"combine" + 0.010*"good"'),
 (2,
  '0.150*"game" + 0.090*"fun" + 0.065*"great" + 0.046*"play" + 0.033*"lot" + 0.031*"little" + 0.030*"luck" + 0.025*"go" + 0.023*"light" + 0.022*"nice" + 0.021*"good" + 0.021*"player" + 0.020*"strategy" + 0.016*"quick" + 0.015*"time" + 0.014*"short" + 0.014*"gamer" + 0.013*"change" + 0.011*"say" + 0.010*"pus

Now we check if the NOUN only approach works better.

In [29]:
# Now we try to work on noun only to see if we get a good representation.
# todo: First thing to do is filter the pos_tagged for names
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.csv"
pos_tagged_ds = pd.read_csv(corpus_path)['comments']

ds = pos_tagged_ds.swifter.apply(lambda x: x.split(' '))
ds = ds.swifter.apply(lambda x: [w.split('__noun')[0] for w in x if w.endswith('__noun')])
ds = ds[ds.map(len) > 1].map(lambda x: ' '.join(x)).drop_duplicates()

corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
ds.to_csv(corpus_path, index=False)

Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

In [32]:
# Now we try to work on noun only to see if we get a good representation.
# todo: First thing to do is filter the pos_tagged for names
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.test.csv"
pos_tagged_ds = pd.read_csv(test_corpus_path)['comments']

ds = pos_tagged_ds.swifter.apply(lambda x: x.split(' '))
ds = ds.swifter.apply(lambda x: [w.split('__noun')[0] for w in x if w.endswith('__noun')])
ds = ds[ds.map(len) > 1].map(lambda x: ' '.join(x)).drop_duplicates()

test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"
ds.to_csv(test_corpus_path, index=False)

Pandas Apply:   0%|          | 0/20234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20234 [00:00<?, ?it/s]

In [30]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

default_config = LdaGeneratorConfig(corpus_file_path=corpus_path)
lda, dictionary = LdaModelGenerator(default_config).make_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/52120 [00:00<?, ?it/s]

In [33]:
from gensim.models import CoherenceModel
import pandas as pd

test_ds = pd.read_csv(test_corpus_path)

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
coh = CoherenceModel(lda, texts=texts, coherence='c_v', topn=topn)
print(f"On topn={topn} we have:")
print(f"Model coherence of: {coh.get_coherence()}")
print("Coherence per topic:")
coh.get_coherence_per_topic()

On topn=10 we have:
Model coherence of: 0.5132157496905211
Coherence per topic:


[0.42519634653314925,
 0.43818274737546564,
 0.539677488166524,
 0.5553321656515858,
 0.3293328420700734,
 0.7444659041518065,
 0.418483283383391,
 0.701529084827182,
 0.6136372814428978,
 0.46103046904622724,
 0.5133169420791968,
 0.48105071346089795,
 0.4076378102582009,
 0.556147417220697]

In [49]:
lda.show_topics(20, 20)

[(0,
  '0.170*"game" + 0.144*"rule" + 0.102*"play" + 0.059*"rating" + 0.030*"lot" + 0.020*"mode" + 0.019*"system" + 0.016*"brain" + 0.016*"battle" + 0.016*"series" + 0.015*"head" + 0.015*"designer" + 0.014*"enemy" + 0.013*"book" + 0.012*"depth" + 0.012*"rulebook" + 0.011*"wargame" + 0.011*"way" + 0.011*"house" + 0.011*"company"'),
 (1,
  '0.148*"game" + 0.143*"<game_name>" + 0.104*"version" + 0.097*"worker" + 0.095*"placement" + 0.074*"strategy" + 0.062*"art" + 0.027*"euro" + 0.024*"work" + 0.022*"idea" + 0.022*"taste" + 0.018*"aspect" + 0.016*"randomness" + 0.015*"half" + 0.014*"memory" + 0.014*"reason" + 0.011*"racing" + 0.011*"mechanism" + 0.010*"fight" + 0.008*"chess"'),
 (2,
  '0.074*"game" + 0.071*"collection" + 0.057*"map" + 0.048*"space" + 0.047*"board" + 0.038*"bonus" + 0.034*"weight" + 0.033*"drafting" + 0.028*"price" + 0.024*"addition" + 0.023*"word" + 0.021*"world" + 0.021*"trade" + 0.020*"improvement" + 0.020*"train" + 0.019*"wife" + 0.017*"text" + 0.017*"change" + 0.016*"

## First default approach

In [1]:
from main.lda.model import LdaModelGenerator, LdaGeneratorConfig

corpus = "../../output/ds/pre_processed.80k.csv"

config = LdaGeneratorConfig(corpus_file_path=corpus)
generator = LdaModelGenerator(config)

In [2]:
base_model, dictionary = generator.make_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

## Evaluation

In [3]:
import pandas as pd
from main.lda.dataset import LdaDataset
from gensim.models import CoherenceModel

topn = 25  # For the coherence evaluation.
test_corpus = "../../output/ds/pre_processed.80k.csv"

In [4]:
test_ds = pd.read_csv(config.corpus_file_path)

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

# For the u_mass model
lda_dataset = LdaDataset(test_ds).dataset

coh_model = CoherenceModel(base_model, corpus=lda_dataset, coherence='u_mass', topn=topn)
coh = CoherenceModel(base_model, texts=texts, coherence='c_v', topn=topn)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

In [16]:
coh.get_coherence_per_topic()

[0.36597146233929445,
 0.27384931723728334,
 0.22480580524809982,
 0.30050388787067583,
 0.32920690002253994,
 0.3230491979414686,
 0.2341942312907988,
 0.25688931688541866,
 0.1910254904136819,
 0.3386633302787434,
 0.29873448478566844,
 0.5631769278233683,
 0.26966071963374144,
 0.22413400542008258]

In [8]:
coh_model.get_coherence()

-5.430695077171264

In [10]:
base_model.log_perplexity(lda_dataset.dataset)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

-10.078125681065158

In [ ]:
# Now the model has high coherence which is good but are the extracted aspects good?
# Probably not.

In [26]:
base_model.show_topic(6)

[('expansion', 0.04917987),
 ('score', 0.048671924),
 ('mean', 0.029798623),
 ('similar', 0.029067513),
 ('couple', 0.024561541),
 ('read', 0.022952618),
 ('drafting', 0.020841701),
 ('provide', 0.020252705),
 ('campaign', 0.02017491),
 ('issue', 0.019857872)]

In [ ]:
topic_distribution = base_model.get_document_topics(dictionary.doc2bow("game hard slow time"))
dominant_topic = max(topic_distribution, key=lambda t: t[1])[0]

In [7]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(base_model, lda_dataset, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.160220  0.101462       1        1  10.228478
11     0.091424 -0.293784       2        1   9.917435
4     -0.145752  0.030663       3        1   9.293434
1     -0.147402 -0.149127       4        1   9.036471
10    -0.180771 -0.065883       5        1   7.968856
12    -0.132376 -0.140732       6        1   7.432195
0      0.005436  0.198822       7        1   7.003833
8      0.082123  0.148314       8        1   6.786641
3     -0.135015  0.068571       9        1   6.587917
7     -0.125794  0.050905      10        1   6.147584
9      0.256626  0.008942      11        1   5.633122
13     0.166008  0.071100      12        1   4.893969
6      0.148035  0.065646      13        1   4.622214
2      0.277679 -0.094898      14        1   4.447850, topic_info=             Term          Freq         Total Category  logprob  loglift
49    <game_name>  16476.000000  16476.000000  Default  30.0000  30.0000
87           card  11371.000000  11371.000000  Default  29.0000  29.0000
64           like  10228.000000  10228.000000  Default  28.0000  28.0000
80           time   6406.000000   6406.000000  Default  27.0000  27.0000
13           play  22382.000000  22382.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
451         fresh    256.899900    257.989574  Topic14  -4.8039   3.1085
511       benefit    230.881864    231.971536  Topic14  -4.9106   3.1080
1453    potential    228.337436    229.427108  Topic14  -4.9217   3.1080
1834       unlike    216.420760    217.510433  Topic14  -4.9753   3.1077
422     different   1770.159036   3017.261644  Topic14  -2.8737   2.5795

[577 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
49        8  0.999950  <game_name>
156       4  0.267906     <number>
156       5  0.731441     <number>
470      12  0.999250      ability
476      12  0.998004         able
...     ...       ...          ...
817       4  0.998379        write
1101      3  0.995907        wrong
1783     12  0.998287            x
242       4  0.998900         year
604       3  0.994740          yes

[637 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 12, 5, 2, 11, 13, 1, 9, 4, 8, 10, 14, 7, 3])